In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_csv('global_power_plant_database.csv')

/tmp/ipykernel_5604/3325319819.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('global_power_plant_database.csv')


In [3]:
countries = ["United States of America", "United Kingdom"]

In [4]:
df_subset = df[df['country_long'].isin(countries)]

In [5]:
# Save to new CSV
df_subset.to_csv("gppd_subset_us_uk.csv", index=False)

In [6]:
def safe_row_to_passage(row):
    def s(val):
        if pd.isna(val) or val in [0, "0", "nan", "NaN"]:
            return None
        # if it's a float with decimal but near a whole number
        if isinstance(val, float):
            if val.is_integer():
                return str(int(val))
            else:
                # handle weird values like 2013.77551 -> round
                return str(int(round(val)))
        return str(val).strip()

    name = s(row.get("name"))
    country = s(row.get("country_long") or row.get("country"))
    primary = s(row.get("primary_fuel"))
    capacity = s(row.get("capacity_mw"))
    year = s(row.get("commissioning_year"))
    lat = s(row.get("latitude"))
    lon = s(row.get("longitude"))

    parts = []
    if name and country:
        parts.append(f"The {name} power plant in {country}.")
    elif name:
        parts.append(f"The {name} power plant.")
    if primary or capacity:
        primary_part = f"It is a {primary} facility" if primary else "Facility"
        capacity_part = f" with a capacity of {capacity} MW" if capacity else ""
        parts.append(primary_part + capacity_part + ".")
    if year:
        parts.append(f"It was commissioned in {year}.")
    if lat and lon:
        parts.append(f"Located at latitude {lat} and longitude {lon}.")

    # join into one tidy passage
    return " ".join(parts)


In [7]:
df_subset

,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,...,estimated_generation_gwh_2013,estimated_generation_gwh_2014,estimated_generation_gwh_2015,estimated_generation_gwh_2016,estimated_generation_gwh_2017,estimated_generation_note_2013,estimated_generation_note_2014,estimated_generation_note_2015,estimated_generation_note_2016,estimated_generation_note_2017
21959,GBR,United Kingdom,14 Tullywiggan Road,GBR0005673,1.000,54.6221,-6.7398,Biomass,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION
21960,GBR,United Kingdom,289 Finvoy Road,GBR0004884,5.900,54.9638,-6.4930,Solar,NaN,NaN,...,8.85,8.83,8.88,8.88,8.95,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE
21961,GBR,United Kingdom,30 Acres,GBR0005396,4.965,53.2062,-1.1968,Solar,NaN,NaN,...,7.09,7.55,7.59,7.61,7.69,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE
21962,GBR,United Kingdom,A C Shropshire (Farm AD),GBR0000233,2.500,52.5860,-1.2375,Biomass,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION
21963,GBR,United Kingdom,A'Chruach,GBR0004331,42.600,56.1528,-5.3102,Wind,NaN,NaN,...,NaN,NaN,NaN,NaN,107.39,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34538,USA,United States of America,Zion Landfill Gas to Energy Facility,USA0056871,7.000,42.4803,-87.8861,Waste,NaN,NaN,...,NaN,NaN,NaN,NaN,1.26,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1
34539,USA,United States of America,Zorn,USA0001368,18.000,38.2803,-85.7023,Gas,NaN,NaN,...,NaN,NaN,NaN,NaN,63.47,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1
34540,USA,United States of America,Zotos International WPGF,USA0057648,3.400,42.8869,-76.9683,Wind,NaN,NaN,...,8.31,8.46,6.57,6.52,6.81,WIND-V1,WIND-V1,WIND-V1,WIND-V1,WIND-V1
34541,USA,United States of America,Zumbro Community Solar Garden,USA0061574,1.000,44.3195,-92.6703,Solar,NaN,NaN,...,NaN,NaN,NaN,NaN,1.62,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1


In [9]:
# Apply to subset
df_subset['passage'] = df_subset.apply(safe_row_to_passage, axis=1)



# Save processed text dataset
df_subset[["country_long", "name", "primary_fuel", "capacity_mw", "commissioning_year", "passage"]].to_csv("gppd_passages_us_uk.csv", index=False)

/tmp/ipykernel_5604/3265926863.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['passage'] = df_subset.apply(safe_row_to_passage, axis=1)


In [16]:
df_check = pd.read_csv("gppd_passages_us_uk.csv")
df_check.head(10)


FileNotFoundError: [Errno 2] No such file or directory: 'gppd_passages_us_uk.csv'

In [ ]:
df["commissioning_year"] = (
    df["commissioning_year"]
    .round(0)                # round to nearest integer
    .astype("Int64")         # use pandas nullable integer (keeps NaN)
)

In [11]:
os.rename("gppd_passages_us_uk.csv", "data.csv")


In [12]:
df = pd.read_csv("data.csv")

In [15]:
df

,country_long,name,primary_fuel,capacity_mw,commissioning_year,passage
0,United Kingdom,14 Tullywiggan Road,Biomass,1.000,<NA>,The 14 Tullywiggan Road power plant in United ...
1,United Kingdom,289 Finvoy Road,Solar,5.900,<NA>,The 289 Finvoy Road power plant in United King...
2,United Kingdom,30 Acres,Solar,4.965,<NA>,The 30 Acres power plant in United Kingdom. It...
3,United Kingdom,A C Shropshire (Farm AD),Biomass,2.500,<NA>,The A C Shropshire (Farm AD) power plant in Un...
4,United Kingdom,A'Chruach,Wind,42.600,<NA>,The A'Chruach power plant in United Kingdom. I...
...,...,...,...,...,...,...
12579,United States of America,Zion Landfill Gas to Energy Facility,Waste,7.000,2003,The Zion Landfill Gas to Energy Facility power...
12580,United States of America,Zorn,Gas,18.000,1969,The Zorn power plant in United States of Ameri...
12581,United States of America,Zotos International WPGF,Wind,3.400,2012,The Zotos International WPGF power plant in Un...
12582,United States of America,Zumbro Community Solar Garden,Solar,1.000,2018,The Zumbro Community Solar Garden power plant ...
